In [136]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import sys
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
src_path = os.environ.get('PROJECT_SRC')
os.chdir(src_path)

In [5]:
import os

# Specify the directory path you want to set as the working directory
directory_path =  '/Users/rachm/PyPSA-GB/Dissertation'

# Change the current working directory to the specified directory path
try:
    os.chdir(directory_path)
    print(f"Current working directory changed to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory at {directory_path} was not found.")

Current working directory changed to: C:\Users\rachm\PyPSA-GB\Dissertation


In [142]:
import requests
from datetime import datetime, timedelta
import csv
import calendar

# Elexon API endpoint and API key
api_url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/pricing/market-index'
api_key = ''  # Replace with your actual API key

# Function to fetch data for a specific date range
def fetch_data_for_month(year, month):
    # Determine the number of days in the month
    num_days = calendar.monthrange(year, month)[1]

    # Initialize variables
    data_all = []

    # Headers
    headers = {
        'accept': 'text/csv',
        'x-api-key': api_key}

    # Iterate over each week of the month
    current_date = datetime(year, month, 1)
    while current_date.month == month:
        end_date = current_date + timedelta(days=6)
        if end_date.month != month:
            end_date = datetime(year, month, num_days)

        params = {
            'from': current_date.strftime('%Y-%m-%dT00:00Z'),
            'to': end_date.strftime('%Y-%m-%dT23:59Z'),
            'dataProviders': 'N2EXMIDP',
            'format': 'csv'}
        try:
            # Make GET request
            response = requests.get(api_url, params=params, headers=headers)

            # Check if request was successful
            if response.status_code == 200:
                # Append the fetched data to the list
                data_all.append(response.text)
            else:
                print(f"Error fetching data for {current_date.date()} to {end_date.date()}: {response.status_code} - {response.text}")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {current_date.date()} to {end_date.date()}: {e}")
        current_date = end_date + timedelta(days=1)
    return data_all

# Function to save data to CSV file
def save_to_csv(data, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        header_written = False
        for block in data:
            lines = block.splitlines()
            reader = csv.reader(lines)
            header = next(reader)
            if not header_written:
                writer.writerow(header)
                header_written = True
            for row in reader:
                writer.writerow(row)

# Example: Fetch data for January 2017 and save to CSV file
year = 2017
month = 1
data_for_month = fetch_data_for_month(year, month)

# Specify the filename to save
csv_filename = f'market_index_data_{year}_{month}.csv'

# Save data to CSV file
save_to_csv(data_for_month, csv_filename)

print(f"Data saved to {csv_filename}")

Data saved to market_index_data_2016_1.csv
